# `My practice` work on PySpark

In [2]:
# создание SparkSession
from pyspark.sql import SparkSession
from fields import *

spark = SparkSession.builder.appName("My_practice_1-file_3").getOrCreate()
# проверяем запуск спарка: 
spark

21/12/06 13:13:46 WARN Utils: Your hostname, vostrolpt resolves to a loopback address: 127.0.1.1; using 172.21.112.248 instead (on interface eth0)
21/12/06 13:13:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/06 13:13:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


`builder` - строительство сессии;
`getOrCreate` - 
`appName` - 

# Четение данных (текст)

In [3]:
spark.read.format('csv').load('data/cars.csv')

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string]

`read` - метод чтения спарком
`format` - задача формата 
`load` - задача пути (откуда читать)

In [4]:
# добавим использование метода(модификатор) spark.read.format().optiom("typeValue", "boolValue")
df = spark.read.format('csv').option(
    'header', 'true').load('data/cars.csv')
df

DataFrame[manufacturer_name: string, model_name: string, transmission: string, color: string, odometer_value: string, year_produced: string, engine_fuel: string, engine_has_gas: string, engine_type: string, engine_capacity: string, body_type: string, has_warranty: string, state: string, drivetrain: string, price_usd: string, is_exchangeable: string, location_region: string, number_of_photos: string, up_counter: string, feature_0: string, feature_1: string, feature_2: string, feature_3: string, feature_4: string, feature_5: string, feature_6: string, feature_7: string, feature_8: string, feature_9: string, duration_listed: string]

# Смотрим на данные в df

.show() - покажет все таблицы
у этого метода есть параметр (vertical)
Дополнительный параметр `vertical=True` выведет каждую строку данных построчно в виде `колонка | значение`

перед самим параметром можно установить кол-во строк под вывод

In [5]:
df.show(1, vertical=True)
# print(type(df))

21/12/06 13:14:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

# DataFrame API

## .select()
это уже очень похожий на стандартный sql-запрос -> код

In [6]:
# я указываю имена полей переменными в качестве аргументов метода .select(), 
# потому что они есть в отдельном питонячем файле fields.py
# в противном случае, поля нужно писать в строке, зная каждое из них

df.select(manufacturer_name, model_name).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



### Другой вариант обращения, по аналогии с pandas

In [7]:
# передача по ключу, как в pandas
df.select(df[manufacturer_name], df[model_name]).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



### или, используя пакет function, а именно методом functions.col


In [8]:
import pyspark.sql.functions as F
df.select(F.col(manufacturer_name), F.col(model_name)).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



# .filter()
как писать условие фильтрации возврата запросов

In [9]:
# отфилтруем только машины марки Subaru
df.select(manufacturer_name, model_name,
        transmission, state).filter(
        "manufacturer_name='Subaru'").show(5)


# df.select(manufacturer_name, color,
#         engine_fuel, state).filter(
#                 "engine_fuel='gasoline'\
#                 and\
#                 state != 'owned'\
#                 and state != 'new'").show(300)

+-----------------+----------+------------+-----+
|manufacturer_name|model_name|transmission|state|
+-----------------+----------+------------+-----+
|           Subaru|   Outback|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
|           Subaru|  Forester|   automatic|owned|
|           Subaru|   Impreza|  mechanical|owned|
|           Subaru|    Legacy|   automatic|owned|
+-----------------+----------+------------+-----+
only showing top 5 rows



можно писать филтры цепочкой

In [10]:
# оба варианта вернут одинаковый результат 

# пишем по аналогии с sql условных операторов 
df_and = df.select(
    manufacturer_name, model_name, transmission, state)\
        .filter(
            "manufacturer_name='Subaru'\
            AND\
            transmission='automatic'").show(5)

# чередовать сами фильтры через точку
df_dot = df.select(
    manufacturer_name, model_name, transmission, state)\
        .filter(
            "manufacturer_name='Subaru'")\
        .filter("transmission='automatic'")\
        .show(5)

+-----------------+----------+------------+-----+
|manufacturer_name|model_name|transmission|state|
+-----------------+----------+------------+-----+
|           Subaru|   Outback|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
|           Subaru|  Forester|   automatic|owned|
|           Subaru|    Legacy|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
+-----------------+----------+------------+-----+
only showing top 5 rows

+-----------------+----------+------------+-----+
|manufacturer_name|model_name|transmission|state|
+-----------------+----------+------------+-----+
|           Subaru|   Outback|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
|           Subaru|  Forester|   automatic|owned|
|           Subaru|    Legacy|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
+-----------------+----------+------------+-----+
only showing top 5 rows



### теперь рассмотрим весьма удобный способ чередования условий через метод .col()

In [11]:
# пример 1
df_dot = df.select(
    manufacturer_name, model_name, transmission, state)\
        .filter(F.col(manufacturer_name) == 'Subaru')\
        .filter(F.col(transmission) =='automatic')\
        .show(5)

+-----------------+----------+------------+-----+
|manufacturer_name|model_name|transmission|state|
+-----------------+----------+------------+-----+
|           Subaru|   Outback|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
|           Subaru|  Forester|   automatic|owned|
|           Subaru|    Legacy|   automatic|owned|
|           Subaru|   Outback|   automatic|owned|
+-----------------+----------+------------+-----+
only showing top 5 rows



Почему метод .col() наиболее предпочтительнее? :
а) питонячий синтаксис;
б) передача параметров для условий фильтрации - может прихожть из `вне`, и передаваться
    в условия дефой / методом (смотри `пример 2`)
в) 

In [12]:
# пример 2
subick = 'Subaru' # вот тут задали значение

# df_dot = df.select(
#     manufacturer_name, model_name, transmission, state)\
#         .filter(F.col(manufacturer_name) == subick).show(5)

df_dot = df.select(
    manufacturer_name,
    engine_has_gas,
    transmission,
    state)\
        .filter(
            F.col(manufacturer_name)== subick)\
            .filter(F.col(
                transmission) == 'automatic')\
                    .filter(
                        F.col(state) != 'owned')\
                            .show(5)

        
# кроме того - наши переменные из fields.py становятся явно указываемыми 
# (ссылающимися) именно в нужный конфиг, от куда их импортировали - только в этом варианте


+-----------------+--------------+------------+---------+
|manufacturer_name|engine_has_gas|transmission|    state|
+-----------------+--------------+------------+---------+
|           Subaru|         False|   automatic|emergency|
|           Subaru|         False|   automatic|emergency|
|           Subaru|         False|   automatic|emergency|
|           Subaru|         False|   automatic|emergency|
+-----------------+--------------+------------+---------+



### метод .count()


In [13]:
df.count()

38531

In [14]:
# для того, что бы исключить дубли
all_dtnc = df.distinct().count()
# --> вернет общее кол-во, т.к. все колонки уникальны: 38531
filter_dtnc = df.select(manufacturer_name).distinct().count()
# --> тут вернет уже отфилтрованное кол-во колонок : 55

print(f'all_dtnc = {all_dtnc}, filter_dtnc = {filter_dtnc}')



all_dtnc = 38491, filter_dtnc = 55


### .groupBy() и .orderBy()

группировка и агрегация

In [15]:
# группировка выполняется аналогично SQL запросу
# посчитаем кол-во каждой марки (упоминание ее в таблице)
df.groupBy(manufacturer_name).count().show(5)
df.groupBy(transmission).count().show(5)
df.groupBy(state).count().show(5)

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|            Lexus|  213|
|           Jaguar|   53|
|            Rover|  235|
|           Lancia|   92|
+-----------------+-----+
only showing top 5 rows

+------------+-----+
|transmission|count|
+------------+-----+
|   automatic|12898|
|  mechanical|25633|
+------------+-----+

+---------+-----+
|    state|count|
+---------+-----+
|      new|  438|
|emergency|  370|
|    owned|37723|
+---------+-----+



`сортировка` по колонке возрастания

In [18]:
# по возрастанию 
df.groupBy(manufacturer_name).count().orderBy(F.col("count")).show(5)

# df.groupBy(manufacturer_name).count().orderBy(F.col(manufacturer_name)).show(5)
# print("--------------------------------------\n")

# по убыванию
df.groupBy(manufacturer_name).count().orderBy(F.col("count").desc()).show(5)

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|          Lincoln|   36|
|       Great Wall|   36|
|          Pontiac|   42|
|              ЗАЗ|   42|
|         Cadillac|   43|
+-----------------+-----+
only showing top 5 rows

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows



# .withColumnRanamed and .withColumn()

### Переименование колонок:

#### Rename-колонки. 1й аргумент == название текущей колонки, 2й аргумент == название новой колонки : 


In [22]:
df.withColumnRenamed(manufacturer_name, "manufactured").show(5, vertical=True)
# or
# df.withColumnRenamed(manufacturer_name, "manufactured").select("manufactured", transmission).show()

-RECORD 0---------------------------
 manufactured     | Subaru          
 model_name       | Outback         
 transmission     | automatic       
 color            | silver          
 odometer_value   | 190000          
 year_produced    | 2010            
 engine_fuel      | gasoline        
 engine_has_gas   | False           
 engine_type      | gasoline        
 engine_capacity  | 2.5             
 body_type        | universal       
 has_warranty     | False           
 state            | owned           
 drivetrain       | all             
 price_usd        | 10900.0         
 is_exchangeable  | False           
 location_region  | Минская обл.    
 number_of_photos | 9               
 up_counter       | 13              
 feature_0        | False           
 feature_1        | True            
 feature_2        | True            
 feature_3        | True            
 feature_4        | False           
 feature_5        | True            
 feature_6        | False           
 

#### Создание новой колонки. 1й аргумент == название новой колонки, 2й аргумент == выражение (обязательно использовать `col()`, если ссылаемся на другую колокну): 

In [27]:
df.withColumn("next_year", F.col(year_produced) + 1).select(year_produced, "next_year").show()

+-------------+---------+
|year_produced|next_year|
+-------------+---------+
|         2010|   2011.0|
|         2002|   2003.0|
|         2001|   2002.0|
|         1999|   2000.0|
|         2001|   2002.0|
|         2011|   2012.0|
|         1998|   1999.0|
|         2004|   2005.0|
|         2010|   2011.0|
|         1999|   2000.0|
|         2003|   2004.0|
|         2008|   2009.0|
|         2002|   2003.0|
|         2001|   2002.0|
|         2011|   2012.0|
|         2004|   2005.0|
|         2005|   2006.0|
|         2005|   2006.0|
|         2014|   2015.0|
|         2013|   2014.0|
+-------------+---------+
only showing top 20 rows



# printSchema() and describe()

In [29]:
# посмотреть как спарк разобра колонки дата-фрейма и к чему привел в процессе работы
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)


#### Для вывода сводной статистики используется `.describe()`:

In [38]:
# df.describe().show(vertical=True)
# or - specific column/columns
df.select(manufacturer_name, transmission, year_produced).describe().show()

+-------+-----------------+------------+------------------+
|summary|manufacturer_name|transmission|     year_produced|
+-------+-----------------+------------+------------------+
|  count|            38531|       38531|             38531|
|   mean|             null|        null|2002.9437336170874|
| stddev|             null|        null| 8.065730511309367|
|    min|            Acura|   automatic|              1942|
|    max|              УАЗ|  mechanical|              2019|
+-------+-----------------+------------+------------------+



In [39]:
spark.stop()